In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import dask.dataframe as dd
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/ubiquant-market-prediction/train.csv',nrows = 200000)
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
submission = pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train = train.drop(columns = ['row_id','time_id','investment_id'],axis = 1)
test1 = test.drop(columns = ['row_id','time_id','investment_id'],axis = 1)

In [ ]:
X=train.drop('target',axis=1)
y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, random_state=101, test_size=0.1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [ ]:
X_train_std= sc.fit_transform(X_train)
test_std= sc.transform(test1)
X_val_std= sc.transform(X_val)

In [ ]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()

In [ ]:
lr.fit(X_train_std,y_train)

In [ ]:
Y_pred_lr=lr.predict(X_val_std)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(r2_score(y_val,Y_pred_lr))
print(mean_absolute_error(y_val,Y_pred_lr))
print(np.sqrt(mean_squared_error(y_val,Y_pred_lr)))

In [ ]:
from lightgbm import *
from sklearn.model_selection import *

In [ ]:
model = LGBMRegressor(
        objective="regression",
        metric="rmse",
        boosting_type="gbdt",
        n_estimators=1000,
        num_leaves=100,
        max_depth=25,
        learning_rate=0.01,
        subsample=0.8
)

In [ ]:
model.fit(X_train_std,y_train,
          eval_set=[(X_val_std, y_val)],
          eval_metric='rmse',
          callbacks=[early_stopping(10), log_evaluation(10)])

In [ ]:
Y_pred=model.predict(X_val_std)

In [ ]:
print(r2_score(y_val,Y_pred))
print(mean_absolute_error(y_val,Y_pred))
print(np.sqrt(mean_squared_error(y_val,Y_pred)))

In [ ]:
y_pred=model.predict(test1)

In [ ]:
y_pred